# Aurora Read Replica Auto Scaling Example


<div style="background-color: #f8f9fa; border: 1px solid #e9ecef; border-radius: 8px; padding: 10px; margin: 10px;">
<strong>📋 Workshop Contents</strong>
<ul style="line-height: 1.2;">
<li><a href="#What-We'll-Build">What We'll Build</a></li>
<li><a href="#Prerequisites">Prerequisites</a></li>
<li><a href="#Step-1:-Configure-Aurora-Auto-Scaling">Step 1: Configure Aurora Auto Scaling</a></li>
<li><a href="#Step-2:-Generate-CPU-Intensive-Workload">Step 2: Generate CPU Intensive Workload</a></li>
<li><a href="#Step-3:-Monitor-Auto-Scaling-Events">Step 3: Monitor Auto Scaling Events</a></li>
<li><a href="#Creating-Read-Replicas">Creating Read Replicas</a></li>
<li><a href="#Connection-Routing-and-Load-Balancing">Connection Routing and Load Balancing</a></li>
<li><a href="#Additional-Resources">Additional Resources</a></li>
</ul>
</div>

## What We'll Build

This code demonstrates Aurora auto scaling with read replicas by:
- Configuring Aurora auto scaling with low CPU thresholds for testing
- Generating CPU-intensive workload using pgbench
- Monitoring auto scaling events and metrics
- Creating Aurora read replicas for load distribution
- Showing connection routing between writer and reader endpoints

## Prerequisites

Before starting this workshop, ensure you have:

- ✅ **Jupyter Notebook**: You can launch a [free tier Amazon SageMaker Jupyter Notebook](../../1_Getting_Started_with_AWS/1.4_Setting_up_Your_Cookbook_Environment/README.MD)
- ✅ **Aurora PostgreSQL Cluster**: An active Aurora PostgreSQL cluster running in your AWS account
  - If you don't have one, follow the setup guide: [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD)
- ✅ **Network Connectivity**: Ensure your Jupyter notebook can connect to the Amazon Aurora cluster
  - Verify security groups allow connections from the Jupyter notebook, for example, check `telnet <Aurora cluster endpoint> <database port>` from your Jupyter notebook's terminal.
  - Database should be accessible from the VPC where the notebook is running.
- ✅ **Database Credentials**: Valid username and password for the Aurora cluster
- ✅ **IAM Permissions** for:
  - Application Auto Scaling (for Aurora auto scaling configuration)
  - RDS cluster management and monitoring
  - CloudWatch metrics access
- ✅ **PostgreSQL Tools**: pgbench installed (included in Amazon SageMaker notebook environment)

> 💡 **Note**: This workshop demonstrates Aurora auto scaling using pgbench load testing. The low CPU thresholds are for demonstration purposes only.

**Important:** Before running this code, update the following variables with your actual values:
- `cluster_id`: Your Aurora cluster identifier
- `db_host`: Your Aurora cluster endpoint
- `db_username`: Database username
- `db_password`: Database password
- `aws_region`: Your AWS region (e.g., "us-east-1")

> 🔗 **Using Previous Aurora Cluster**: If you completed the previous sections, you can reuse the Aurora cluster and credentials from [Your First Database on AWS](../../2_Your_First_Database_on_AWS/README.MD).

In [ ]:
import boto3
import time
import pandas as pd
import subprocess
import os
from IPython.display import display, HTML
from datetime import datetime, timedelta

# Configuration - Update these values
cluster_id = "your-aurora-cluster"
aws_region = "us-east-1"
replica_class = "db.r7g.large"
db_host = "your-cluster-endpoint.cluster-xxx.us-east-1.rds.amazonaws.com"
db_username = "postgres"
db_password = "your-password"
db_name = "postgres"

print("🚀 Aurora Auto Scaling Demo Configuration:")
print(f"Cluster ID: {cluster_id}")
print(f"Region: {aws_region}")
print(f"Instance Class: {replica_class}")

## Step 1: Configure Aurora Auto Scaling

First, we'll configure Aurora auto scaling with a low CPU threshold (20%) for testing purposes. This will trigger scaling events more easily during our load test.

In [ ]:
def configure_aurora_autoscaling(cluster_identifier, region='us-east-1'):
    """Configure Aurora auto scaling with low CPU threshold for testing"""
    autoscaling = boto3.client('application-autoscaling', region_name=region)
    cloudwatch = boto3.client('cloudwatch', region_name=region)
    
    resource_id = f"cluster:{cluster_identifier}"
    
    try:
        # Register scalable target
        print("📊 Registering Aurora cluster as scalable target...")
        autoscaling.register_scalable_target(
            ServiceNamespace='rds',
            ResourceId=resource_id,
            ScalableDimension='rds:cluster:ReadReplicaCount',
            MinCapacity=0,
            MaxCapacity=3,
            Tags=[
                {
                    'Key': 'CreationSource',
                    'Value': 'aws-database-cookbook-v2025.8'
                }
            ]
        )
        
        # Create scaling policy with low CPU threshold
        print("⚙️ Creating scaling policy with 20% CPU threshold...")
        policy_response = autoscaling.put_scaling_policy(
            PolicyName=f"{cluster_identifier}-cpu-scaling-policy",
            ServiceNamespace='rds',
            ResourceId=resource_id,
            ScalableDimension='rds:cluster:ReadReplicaCount',
            PolicyType='TargetTrackingScaling',
            TargetTrackingScalingPolicyConfiguration={
                'TargetValue': 20.0,  # Low threshold for testing
                'PredefinedMetricSpecification': {
                    'PredefinedMetricType': 'RDSReaderAverageCPUUtilization'
                },
                'ScaleOutCooldown': 300,
                'ScaleInCooldown': 300
            },
            Tags=[
                {
                    'Key': 'CreationSource',
                    'Value': 'aws-database-cookbook-v2025.8'
                }
            ]
        )
        
        print("✅ Auto scaling configured successfully!")
        print("📈 Scale-out threshold: 20% CPU utilization")
        print("📉 Scale-in threshold: Below 20% CPU utilization")
        print("🔄 Cooldown period: 5 minutes")
        
        return policy_response['PolicyARN']
        
    except Exception as e:
        print(f"❌ Error configuring auto scaling: {str(e)}")
        return None

# Configure auto scaling
policy_arn = configure_aurora_autoscaling(cluster_id, aws_region)

## Step 2: Generate CPU Intensive Workload

Now we'll use pgbench to generate a CPU-intensive workload that will trigger Aurora auto scaling. We'll run multiple concurrent connections to increase CPU utilization.

In [ ]:
def setup_pgbench_database():
    """Initialize pgbench database for load testing"""
    print("🔧 Setting up pgbench database...")
    
    # Initialize pgbench tables
    init_cmd = [
        'pgbench',
        '-i',  # Initialize
        '-s', '10',  # Scale factor
        '-h', db_host,
        '-U', db_username,
        '-d', db_name
    ]
    
    try:
        env = os.environ.copy()
        env['PGPASSWORD'] = db_password
        
        result = subprocess.run(init_cmd, env=env, capture_output=True, text=True)
        
        if result.returncode == 0:
            print("✅ pgbench database initialized successfully!")
            print("📊 Created tables: pgbench_accounts, pgbench_branches, pgbench_history, pgbench_tellers")
        else:
            print(f"❌ pgbench initialization failed: {result.stderr}")
            
    except FileNotFoundError:
        print("❌ pgbench not found. Install PostgreSQL client tools first.")
        print("💡 Run: sudo apt-get install postgresql-client (Ubuntu) or brew install postgresql (macOS)")

def run_cpu_intensive_workload(duration_minutes=10):
    """Run CPU-intensive pgbench workload"""
    print(f"🚀 Starting CPU-intensive workload for {duration_minutes} minutes...")
    print("📈 This will generate high CPU utilization to trigger auto scaling")
    
    # High-intensity pgbench command
    bench_cmd = [
        'pgbench',
        '-c', '50',  # 50 concurrent clients
        '-j', '10',  # 10 threads
        '-T', str(duration_minutes * 60),  # Duration in seconds
        '-S',  # Select-only transactions (read-heavy)
        '-h', db_host,
        '-U', db_username,
        '-d', db_name
    ]
    
    try:
        env = os.environ.copy()
        env['PGPASSWORD'] = db_password
        
        print("⚡ Running high-concurrency read workload...")
        print("📊 50 concurrent clients, 10 threads, SELECT-only queries")
        
        # Run pgbench in background
        process = subprocess.Popen(bench_cmd, env=env, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        
        return process
        
    except Exception as e:
        print(f"❌ Error running workload: {str(e)}")
        return None

# Setup and start workload
setup_pgbench_database()
workload_process = run_cpu_intensive_workload(10)  # 10 minutes

## Step 3: Monitor Auto Scaling Events

While the workload is running, we'll monitor Aurora auto scaling events and CPU metrics to observe the scaling behavior.

In [ ]:
def monitor_scaling_events(cluster_identifier, region='us-east-1', duration_minutes=10):
    """Monitor Aurora auto scaling events and metrics"""
    rds = boto3.client('rds', region_name=region)
    cloudwatch = boto3.client('cloudwatch', region_name=region)
    autoscaling = boto3.client('application-autoscaling', region_name=region)
    
    print("📊 Monitoring Aurora auto scaling events...")
    print(f"⏱️ Monitoring for {duration_minutes} minutes")
    
    start_time = datetime.utcnow()
    end_time = start_time + timedelta(minutes=duration_minutes)
    
    while datetime.utcnow() < end_time:
        try:
            # Get current cluster status
            cluster_response = rds.describe_db_clusters(DBClusterIdentifier=cluster_identifier)
            cluster = cluster_response['DBClusters'][0]
            instance_count = len(cluster['DBClusterMembers'])
            
            # Get CPU metrics
            cpu_response = cloudwatch.get_metric_statistics(
                Namespace='AWS/RDS',
                MetricName='CPUUtilization',
                Dimensions=[
                    {'Name': 'DBClusterIdentifier', 'Value': cluster_identifier}
                ],
                StartTime=datetime.utcnow() - timedelta(minutes=5),
                EndTime=datetime.utcnow(),
                Period=300,
                Statistics=['Average']
            )
            
            avg_cpu = 0
            if cpu_response['Datapoints']:
                avg_cpu = cpu_response['Datapoints'][-1]['Average']
            
            # Get scaling activities
            scaling_response = autoscaling.describe_scaling_activities(
                ServiceNamespace='rds',
                ResourceId=f"cluster:{cluster_identifier}",
                ScalableDimension='rds:cluster:ReadReplicaCount',
                MaxResults=5
            )
            
            current_time = datetime.utcnow().strftime('%H:%M:%S')
            print(f"\r[{current_time}] Instances: {instance_count} | CPU: {avg_cpu:.1f}% | Scaling Events: {len(scaling_response['ScalingActivities'])}", end='', flush=True)
            
            # Show recent scaling events
            for activity in scaling_response['ScalingActivities'][:2]:
                if activity['CreationTime'] > start_time:
                    print(f"🔄 Scaling Event: {activity['Description']} - {activity['StatusCode']}")
            
            time.sleep(30)  # Check every 30 seconds
            
        except Exception as e:
            print(f"❌ Error monitoring: {str(e)}")
            break
    
    print("✅ Monitoring completed!")

def get_final_scaling_summary(cluster_identifier, region='us-east-1'):
    """Display final scaling summary"""
    rds = boto3.client('rds', region_name=region)
    autoscaling = boto3.client('application-autoscaling', region_name=region)
    
    try:
        # Final cluster status
        cluster_response = rds.describe_db_clusters(DBClusterIdentifier=cluster_identifier)
        cluster = cluster_response['DBClusters'][0]
        
        # Recent scaling activities
        scaling_response = autoscaling.describe_scaling_activities(
            ServiceNamespace='rds',
            ResourceId=f"cluster:{cluster_identifier}",
            ScalableDimension='rds:cluster:ReadReplicaCount',
            MaxResults=10
        )
        
        print("📊 Final Aurora Auto Scaling Summary:")
        print(f"Total Instances: {len(cluster['DBClusterMembers'])}")
        print(f"Scaling Activities: {len(scaling_response['ScalingActivities'])}")
        
        if scaling_response['ScalingActivities']:
            print("🔄 Recent Scaling Events:")
            for activity in scaling_response['ScalingActivities'][:5]:
                event_time = activity['CreationTime'].strftime('%H:%M:%S')
                print(f"  [{event_time}] {activity['Description']} - {activity['StatusCode']}")
        
        return cluster
        
    except Exception as e:
        print(f"❌ Error getting summary: {str(e)}")
        return None

# Start monitoring (run this while workload is active)
print("🚀 Starting monitoring - this will run for 10 minutes...")
monitor_scaling_events(cluster_id, aws_region, 10)

# Get final summary
get_final_scaling_summary(cluster_id, aws_region)

## Creating Read Replicas

The following functions demonstrate manual read replica creation and cluster endpoint management. With auto scaling enabled, Aurora will automatically create and remove read replicas based on CPU utilization.

In [ ]:
def create_read_replica(cluster_identifier, replica_identifier, instance_class, region='us-east-1'):
    """Create Aurora read replica for horizontal scaling"""
    rds = boto3.client('rds', region_name=region)
    
    try:
        print(f"Creating read replica: {replica_identifier}")
        print(f"Instance class: {instance_class}")
        
        response = rds.create_db_instance(
            DBInstanceIdentifier=replica_identifier,
            DBInstanceClass=instance_class,
            Engine='aurora-postgresql',
            DBClusterIdentifier=cluster_identifier,
            PubliclyAccessible=False,
            Tags=[
                {
                    'Key': 'CreationSource',
                    'Value': 'aws-database-cookbook-v2025.8'
                }
            ]
        )
        
        print("✅ Read replica creation initiated!")
        print("⏳ This will take 5-10 minutes to complete...")
        return response
        
    except Exception as e:
        print(f"❌ Error creating read replica: {str(e)}")
        return None

def get_cluster_endpoints(cluster_identifier, region='us-east-1'):
    """Display cluster endpoints for read/write routing"""
    rds = boto3.client('rds', region_name=region)
    
    try:
        response = rds.describe_db_clusters(DBClusterIdentifier=cluster_identifier)
        cluster = response['DBClusters'][0]
        
        endpoints = {
            'Endpoint Type': ['Writer Endpoint', 'Reader Endpoint'],
            'URL': [
                cluster.get('Endpoint', 'N/A'),
                cluster.get('ReaderEndpoint', 'N/A')
            ],
            'Purpose': [
                'Write operations (INSERT, UPDATE, DELETE)',
                'Read operations (SELECT) - Load balanced'
            ]
        }
        
        df = pd.DataFrame(endpoints)
        display(HTML(df.to_html(index=False)))
        
        print(f"📊 Cluster has {len(cluster['DBClusterMembers'])} instances")
        print("🔄 Reader endpoint automatically distributes read traffic")
        
        return cluster
        
    except Exception as e:
        print(f"❌ Error getting cluster info: {str(e)}")
        return None

# Show current cluster configuration
print("Current Cluster Configuration:")
get_cluster_endpoints(cluster_id, aws_region)

## Connection Routing and Load Balancing

After auto scaling creates read replicas, Aurora automatically load balances read traffic across all available read replicas using the reader endpoint. The writer endpoint always routes to the primary instance for write operations.

## Cleanup Resources

After testing, clean up the auto scaling configuration and any additional read replicas to avoid unnecessary costs.

In [ ]:
def cleanup_autoscaling(cluster_identifier, region='us-east-1'):
    """Remove auto scaling configuration"""
    autoscaling = boto3.client('application-autoscaling', region_name=region)
    
    try:
        resource_id = f"cluster:{cluster_identifier}"
        
        # Delete scaling policies
        print("🧽 Removing scaling policies...")
        autoscaling.delete_scaling_policy(
            PolicyName=f"{cluster_identifier}-cpu-scaling-policy",
            ServiceNamespace='rds',
            ResourceId=resource_id,
            ScalableDimension='rds:cluster:ReadReplicaCount'
        )
        
        # Deregister scalable target
        print("🧽 Deregistering scalable target...")
        autoscaling.deregister_scalable_target(
            ServiceNamespace='rds',
            ResourceId=resource_id,
            ScalableDimension='rds:cluster:ReadReplicaCount'
        )
        
        print("✅ Auto scaling configuration removed!")
        
    except Exception as e:
        print(f"❌ Error during cleanup: {str(e)}")

# Uncomment to cleanup (run after testing)
# cleanup_autoscaling(cluster_id, aws_region)

## Usage Instructions

1. **Update Configuration**: Modify the configuration variables at the top with your Aurora cluster details
2. **Run Step 1**: Configure auto scaling with low CPU threshold (20%)
3. **Run Step 2**: Generate CPU-intensive workload using pgbench
4. **Run Step 3**: Monitor auto scaling events in real-time
5. **Observe Results**: Watch as Aurora automatically creates read replicas when CPU exceeds 20%
6. **Cleanup**: Remove auto scaling configuration after testing

**Expected Behavior:**
- CPU utilization will increase due to pgbench workload
- When CPU exceeds 20%, Aurora will create additional read replicas
- Read traffic will be automatically distributed across replicas
- When workload decreases, Aurora will scale in (remove replicas)

**Prerequisites:**
- PostgreSQL client tools (pgbench) installed
- Aurora PostgreSQL cluster running
- Appropriate IAM permissions for RDS and Application Auto Scaling

## Additional Resources 📚

### Aurora Read Replicas & Auto Scaling
- [Aurora Read Replicas](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Replication.html)
- [Aurora Auto Scaling](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Integrating.AutoScaling.html)
- [Read Replica Best Practices](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.BestPractices.html)

### Performance Testing
- [pgbench Documentation](https://www.postgresql.org/docs/current/pgbench.html)
- [Aurora Performance Testing](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/Aurora.Managing.Performance.html)
- [Load Testing Best Practices](https://docs.aws.amazon.com/wellarchitected/latest/performance-efficiency-pillar/test-performance.html)

### Monitoring & Scaling
- [CloudWatch Metrics for Aurora](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/MonitoringAurora.html)
- [Application Auto Scaling](https://docs.aws.amazon.com/autoscaling/application/userguide/what-is-application-auto-scaling.html)
- [Performance Insights](https://docs.aws.amazon.com/AmazonRDS/latest/AuroraUserGuide/USER_PerfInsights.html)